# Create Slice



## 1. Setup and Configuration

In [ ]:
import sys
from pathlib import Path

# Add parent directory to Python path to import modules
repo_root = Path.cwd().parent
# No longer needed - using installed package

print(f"✅ Python path configured")
print(f"   Repository root: {repo_root}")

In [ ]:
import sys
# No longer needed - using installed package  # Add parent directory to path

# Import modules
from fabric_generic_cluster import load_topology_from_yaml_file, SiteTopology
from fabric_generic_cluster import deployment as sd
from fabric_generic_cluster import network_config as snc
from fabric_generic_cluster import ssh_setup as ssh

print("✅ Modules imported successfully")

In [ ]:
# Define YAML directory
YAML_DIR = repo_root / "model"
print(f"✅ YAML directory: {YAML_DIR}")

In [ ]:
# Show Fablib configuration
sd.show_config()

## 2. Check Existing Slices

In [ ]:
sd.check_slices()

## 3. Set Slice Name

In [ ]:
# Define your base slice name
slice_name = "slice-generic-cluster"

# Site topology YAML file
site_topology_yaml = "../model/_slice_topology_8.yaml"

# Define test node(s) and network(s)
source_hostname = "lc-1"
network_name = "net_local_1"
use_ipv6 = False

In [ ]:
# Get a unique version of that name
use_timestamp = False
slice_name = sd.check_or_generate_unique_slice_name(slice_name, use_timestamp)

print(f"✅ Proceeding with slice: {slice_name}")

## 4. Load and Validate Topology


In [ ]:
# Load and validate topology (raises ValidationError if invalid)
try:
    topology = load_topology_from_yaml_file(site_topology_yaml)
    print("✅ Topology loaded and validated successfully!")
    print(f"   Nodes: {len(topology.site_topology_nodes.nodes)}")
    print(f"   Networks: {len(topology.site_topology_networks.networks)}")
except Exception as e:
    print(f"❌ Validation failed: {e}")
    raise

In [ ]:
# Topology file
topology_file_png_name = f"{slice_name}.png"
topology_file_json_name = f"{slice_name}.json"
topology_file_dot_name = f"{slice_name}.dot"

## 5. Explore Topology Using Type-Safe Access


In [ ]:
# Print topology summary using type-safe access
print("\n🔷 Topology Summary:\n")

for node in topology.site_topology_nodes.iter_nodes():
    print(f"Node: {node.hostname}")
    print(f"  Site: {node.site}")
    print(f"  Resources: {node.capacity.cpu} cores, {node.capacity.ram}GB RAM")
    print(f"  OS: {node.capacity.os}")
    
    # Check OpenStack roles
    roles = node.specific.openstack
    active_roles = []
    if roles.is_control(): active_roles.append("control")
    if roles.is_network(): active_roles.append("network")
    if roles.is_compute(): active_roles.append("compute")
    if roles.is_storage(): active_roles.append("storage")
    
    if active_roles:
        print(f"  OpenStack roles: {', '.join(active_roles)}")
    
    # Print network interfaces
    for nic_name, iface_name, iface in node.get_all_interfaces():
        ipv4 = iface.get_ipv4_address() or "No IPv4"
        print(f"  └─ {nic_name}.{iface_name}: {iface.binding} ({ipv4})")
    
    print()

## 6. Visualize Topology


In [ ]:
import sys
# No longer needed - using installed package  # Add parent directory to path

from fabric_generic_cluster import load_topology_from_yaml_file
from fabric_generic_cluster import topology_viewer as viewer

# Load topology
topology = load_topology_from_yaml_file(site_topology_yaml)

# Quick overview
viewer.print_compact_summary(topology)

# Detailed view
viewer.print_topology_summary(topology)

# Visual graph
viewer.draw_topology_graph(topology, show_ip=True, save_path=topology_file_png_name)

# ASCII art
viewer.print_ascii_topology(topology)

# Specific network details
viewer.print_network_details(topology, network_name)

# Export to JSON
viewer.export_topology_to_json(topology, topology_file_json_name )

# Export to Graphviz DOT format
viewer.export_topology_to_dot(topology, topology_file_dot_name )
# Then: dot -Tpng topology.dot -o topology.png


# Color-coded nodes based on OpenStack roles:

# 🟡 Gold = Control nodes
# 🟢 Light green = Compute nodes
# 🟣 Plum = Storage nodes
# 🟢 Green = Other nodes
# 🔵 Blue = Networks

## 7. Deploy Slice to FABRIC


In [ ]:
#
# Deploy
#

try:
    slice = sd.deploy_topology_to_fabric(topology, slice_name)

    print("\n✅ Slice deployment successful!\n")
except Exception as e:
    print(f"\n❌ Slice deployment failed: {e}\n")
    raise

In [ ]:
#
# Configure L3 networks (if connected) 
#

try:
    sd.configure_l3_networks(slice, topology)

    print("\n✅ Configure L3 networks successful!\n")
except Exception as e:
    print(f"\n❌ Configure L3 networks failed: {e}\n")
    raise

In [ ]:
#
# Configure Interfaces (permanent - NetworkManager, Netplan)
#

try:
    snc.configure_node_interfaces(slice, topology)

    print("\n✅ Configure node interfaces successful!\n")
except Exception as e:
    print(f"\n❌ Configure node interfaces failed: {e}\n")
    raise

## 8. Observe Slice Attributes

In [ ]:
try:
    slice = sd.get_slice(slice_name)
    slice.show()
    slice.list_nodes()
    slice.list_networks()
    slice.list_interfaces()
except Exception as e:
    print(f"Exception: {e}")

## 9. Configure Network Interfaces


In [ ]:
# Configure interfaces
snc.configure_node_interfaces(slice, topology)

In [ ]:
# Verify configuration
snc.verify_node_interfaces(slice, topology)

## 10. Setup SSH Environment


In [ ]:
# Complete SSH setup (keys, distribution, config)
success = ssh.setup_passwordless_ssh(slice)

if success:
    print("\n✅ SSH setup completed successfully!\n")
else:
    print("\n⚠️  SSH setup completed with some warnings\n")

In [ ]:
# Update /etc/hosts for hostname resolution
snc.update_hosts_file_on_nodes(slice, topology)

## 11. Test Network Connectivity

In [ ]:
# Test connectivity 

results = snc.ping_network_from_node(
    slice, 
    topology, 
    source_hostname=source_hostname, 
    network_name=network_name,
    use_ipv6=use_ipv6
)

# Check results
if all(results.values()):
    print("\n✅ All ping tests passed!\n")
else:
    print("\n⚠️  Some ping tests failed\n")

## 12. Verify SSH Access

In [ ]:
# Test SSH connectivity

ssh_results = ssh.verify_ssh_access(
    slice,
    topology,
    source_hostname=source_hostname,
    network_name=network_name,
    use_ipv6=use_ipv6
)

if all(ssh_results.values()):
    print("\n✅ All SSH connections successful!\n")
else:
    print("\n⚠️  Some SSH connections failed\n")

## 13. Advanced: Using Type-Safe Queries

Examples of using the new model methods:

In [ ]:
# Find specific node
node = topology.get_node_by_hostname(source_hostname)
print(f"Found node: {node.name} at site {node.site}")

# Get all nodes on a network
nodes_on_net1 = topology.get_nodes_on_network(network_name)
print(f"\nNodes on network_1:")
for n in nodes_on_net1:
    print(f"  - {n.hostname}")

# Get interfaces for a specific network
interfaces = node.get_interfaces_for_network(network_name)
print(f"\n{node.hostname} interfaces on network_1:")
for nic_name, iface in interfaces:
    print(f"  - {nic_name}: {iface.get_ipv4_address()}")

# Check OpenStack roles
if node.specific.openstack.is_control():
    print(f"\n{node.hostname} is a control node")

## 14. Cleanup (Optional)

In [ ]:
# Uncomment to delete the slice
###sd.delete_slice(slice_name)